In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

%matplotlib inline

Scrape the Wikipedia page, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe .

In [37]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
soup = BeautifulSoup(extracting_data, 'lxml')

In [38]:
column_names = ['Postalcode', 'Borough', 'Neighborhood']

gta = pd.DataFrame(columns=column_names)

content =soup.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    gta = gta.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [40]:
gta = gta[gta.Borough!= 0]
gta = gta[gta.Borough != 'Not assigned']
gta.index = range(len(gta))
gta.head(10)


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Combining Neighbourhoods by Postal Code/Borough.

In [41]:
gta = gta.groupby(['Postalcode','Borough']).agg(','.join)
gta = gta.reset_index(level=['Postalcode', 'Borough'])
gta.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Check if there has any Neighbourhoods not assigned.

In [43]:
gtna = gta.loc[gta['Neighborhood'] == 'Not assigned']
gtna

,Postalcode,Borough,Neighborhood


Using the .shape method to print the number of rows of this dataframe.

In [44]:
print(gta.shape)

(103, 3)


In [45]:
gta.to_csv('gta.csv', index=False)